## Travail à faire pour le tableau 2 

1. Extraire les données CDISC STDM utiles au critère principal (LB + DM).

2. Définir le critère de jugement principal :

    - Rétention (présence au jour ≥ 13)

    - Abstinence (test urinaire négatif)

3. Comparer les deux bras de traitement (BUPNAL vs CLON).

4. Construire le Tableau 2 :

    - Effectif

    - Pourcentage

    - Différence de proportions

    - Test statistique

    - Valeur de p

    - IC à 95 %

### Chargement des bibliotheques 

In [1]:
import numpy as np # vecteurs et operations 
import polars as pl #manipulation dataframe 
import scipy as sp # tests statistiques (t-test, test de Levene).

### Chargement des données biologiques (LB) 

In [4]:
labo_hosp = pl.read_csv(r'C:\Users\Abdo\OneDrive\Desktop\M1\Data Challenge 1\ascii-data-files-nida-ctn-0001-20251024\lb.csv')
labo_amb = pl.read_csv(r'C:\Users\Abdo\OneDrive\Desktop\M1\Data Challenge 1\ascii-data-files-nida-ctn-0002-20251027\lb.csv')
lb_hosp_amub = pl.concat([labo_hosp, labo_amb])


## 
Inspection préliminaire

- VISITNUM : contient les numéros de visites (1, 2, 3… 17).

- LBSPEC = "URINE" → OK, c'est bien ce qu’il faut.

- LBTESTCD : tests réalisés (OPIATE, COCAINE, etc.).

- LBORRES : résultat brut (NEGATIVE/POSITIVE).

- LBSTRESC : même chose, version standardisée.

In [5]:
lb_hosp_amub

STUDYID,DOMAIN,USUBJID,EPOCH,LBSEQ,LBTESTCD,LBTEST,LBCAT,LBORRES,LBORRESU,LBORNRLO,LBORNRHI,LBSTRESC,LBSTRESN,LBSTRESU,LBSTNRLO,LBSTNRHI,LBNRIND,LBSTAT,LBREASND,LBSPEC,LBMETHOD,LBBLFL,VISIT,VISITNUM,LBDTC,LBDY
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str
"""NIDA-CTN-0001""","""LB""","""01_000579""","""SCREENING""",1,"""AMPHET""","""AMPHETAMINES""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""ACCUTEST""","""Y""","""BASELINE, ANY DAY PRIOR TO FIR…",0,2001,"""-1"""
"""NIDA-CTN-0001""","""LB""","""01_000579""","""SCREENING""",11,"""AMPHET""","""AMPHETAMINES""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""","""Y""","""BASELINE, ANY DAY PRIOR TO FIR…",0,2001,"""-1"""
"""NIDA-CTN-0001""","""LB""","""01_000579""","""SCREENING""",2,"""BARBIT""","""BARBITURATES""","""DRUG SCREEN""","""POSITIVE""",null,null,null,"""POSITIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""ACCUTEST""","""Y""","""BASELINE, ANY DAY PRIOR TO FIR…",0,2001,"""-1"""
"""NIDA-CTN-0001""","""LB""","""01_000579""","""SCREENING""",3,"""BENZOD""","""BENZODIAZEPINES""","""DRUG SCREEN""","""POSITIVE""",null,null,null,"""POSITIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""ACCUTEST""","""Y""","""BASELINE, ANY DAY PRIOR TO FIR…",0,2001,"""-1"""
"""NIDA-CTN-0001""","""LB""","""01_000579""","""SCREENING""",12,"""BENZOD""","""BENZODIAZEPINES""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""","""Y""","""BASELINE, ANY DAY PRIOR TO FIR…",0,2001,"""-1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NIDA-CTN-0002""","""LB""","""02_099926""","""ACTIVE""",18,"""THC""","""THC""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""STUDY DAY 3""",3,2001,"""3"""
"""NIDA-CTN-0002""","""LB""","""02_099926""","""ACTIVE""",25,"""THC""","""THC""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""STUDY DAY 8""",8,2001,"""8"""
"""NIDA-CTN-0002""","""LB""","""02_099926""","""ACTIVE""",32,"""THC""","""THC""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""STUDY DAY 11""",11,2001,"""11"""


In [6]:
lb_hosp_amub.select(pl.col("VISITNUM").unique())

VISITNUM
i64
-99
0
2
3
4
…
13
14
15


In [7]:
lb_hosp_amub.select(pl.col("LBSPEC").unique())

LBSPEC
str
"""URINE"""


### Hospitaliser 

In [11]:
labo_hosp = labo_hosp.filter(pl.col('EPOCH') == "FOLLOW-UP 3")
labo_hosp

STUDYID,DOMAIN,USUBJID,EPOCH,LBSEQ,LBTESTCD,LBTEST,LBCAT,LBORRES,LBORRESU,LBORNRLO,LBORNRHI,LBSTRESC,LBSTRESN,LBSTRESU,LBSTNRLO,LBSTNRHI,LBNRIND,LBSTAT,LBREASND,LBSPEC,LBMETHOD,LBBLFL,VISIT,VISITNUM,LBDTC,LBDY
str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str
"""NIDA-CTN-0001""","""LB""","""01_000579""","""FOLLOW-UP 3""",21,"""NONE""","""NONE""","""DRUG SCREEN""",null,null,null,null,null,""" """,null,""" """,""" """,null,"""NOT DONE""","""URINE NOT COLLECTED (SPECIFY)""","""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,null,""" """
"""NIDA-CTN-0001""","""LB""","""01_001362""","""FOLLOW-UP 3""",43,"""NONE""","""NONE""","""DRUG SCREEN""",null,null,null,null,null,""" """,null,""" """,""" """,null,"""NOT DONE""","""PATIENT DID NOT ATTEND""","""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,null,""" """
"""NIDA-CTN-0001""","""LB""","""01_001490""","""FOLLOW-UP 3""",43,"""NONE""","""NONE""","""DRUG SCREEN""",null,null,null,null,null,""" """,null,""" """,""" """,null,"""NOT DONE""","""PATIENT DID NOT ATTEND""","""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,null,""" """
"""NIDA-CTN-0001""","""LB""","""01_002199""","""FOLLOW-UP 3""",56,"""NONE""","""NONE""","""DRUG SCREEN""",null,null,null,null,null,""" """,null,""" """,""" """,null,"""NOT DONE""","""PATIENT DID NOT ATTEND""","""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,null,""" """
"""NIDA-CTN-0001""","""LB""","""01_002844""","""FOLLOW-UP 3""",58,"""NONE""","""NONE""","""DRUG SCREEN""",null,null,null,null,null,""" """,null,""" """,""" """,null,"""NOT DONE""","""PATIENT DID NOT ATTEND""","""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,null,""" """
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""NIDA-CTN-0001""","""LB""","""01_097261""","""FOLLOW-UP 3""",47,"""METHADON""","""METHADONE""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,2002,"""184"""
"""NIDA-CTN-0001""","""LB""","""01_097261""","""FOLLOW-UP 3""",48,"""OPIATE""","""OPIATE""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,2002,"""184"""
"""NIDA-CTN-0001""","""LB""","""01_097261""","""FOLLOW-UP 3""",49,"""THC""","""THC""","""DRUG SCREEN""","""NEGATIVE""",null,null,null,"""NEGATIVE""",""" """,null,""" """,""" """,null,null,null,"""URINE""","""CENTRAL LAB""",null,"""FOLLOW-UP 3""",17,2002,"""184"""
